# incremental_enrollment_trial-processing_results
---------

결과 분석

## Environment

In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import pandas as pd
import pickle
import numpy as np
import sys
import os

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
sys.path.append('/host/projects/sv_experiments/sv_system')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
from utils import key2df
from utils import compute_eer

## Results

Meta data는 result.pkl에 저장되고, 

posScore(target sample에 대한 score), negScore(non-target sample에 대한 score), answer(verification output)만을 사용해서 결과를 뽑는다.

### Base result

In [24]:
records = {}
base_dir = "test_out/"
for t_dir in os.listdir(base_dir):
    posScore = pickle.load(open(base_dir+t_dir+'/posScores.pkl', 'rb'))
    negScore = pickle.load(open(base_dir+t_dir+'/negScores.pkl', 'rb'))
    result = pickle.load(open(base_dir+t_dir+'/result.pkl', "rb"))
    n_correct_trials = sum([r[3] for r in result])
    n_total_trials = sum([r[2] for r in result])
    acc = n_correct_trials / n_total_trials
    eer, thresh = compute_eer(posScore, negScore)
    print("[base_{}] eer:{:.4f}, acc:{:.4f}".format(t_dir, eer*100, acc*100))
    records['base_'+t_dir] = (eer, acc)

[base_n_enr_full_pRatio_0.9] eer:3.2733, acc:97.2148


### Incremental system result

In [25]:
inc_records = {}
base_dir = "./test_out/"
for enr in ['3','5','7', '9']: # n_enr cases
    subpath = 'n_enr'+enr+'_results/'
    for n in sorted(os.listdir(base_dir+subpath+'inc/')):
        for t_dir in sorted(os.listdir(base_dir+subpath+'inc/'+ n)):
            posScore = pickle.load(open(base_dir+subpath+'inc/'+n+'/'+t_dir+'/posScores.pkl', 'rb'))
            negScore = pickle.load(open(base_dir+subpath+'inc/'+n+'/'+t_dir+'/negScores.pkl', 'rb'))
            result = pickle.load(open(base_dir+t_dir+'/result.pkl', "rb"))
            n_correct_trials = sum([r[3] for r in result])
            n_total_trials = sum([r[2] for r in result])
            acc = n_correct_trials / n_total_trials
            eer, thresh = compute_eer(posScore, negScore)
            print('inc_{}: eer: {:.5f}, acc: {:.5f}'.format(t_dir, eer*100, acc*100))
            inc_records[n+'_inc_'+t_dir] = (eer, acc)

FileNotFoundError: [Errno 2] No such file or directory: './test_out/n_enr3_results/inc/'

### Write results to txt

In [27]:
eer_string = ''
acc_string = ''
#for trial in ['1','2','3','4','5']:
for trial in ['1']:
    eer_string += 'Q3\tQ5\tQ7\tQ9\tQ3\tQ5\tQ7\tQ9\tQ3\tQ5\tQ7\tQ9\tQ3\tQ5\tQ7\tQ9\n'
    acc_string += 'Q3\tQ5\tQ7\tQ9\tQ3\tQ5\tQ7\tQ9\tQ3\tQ5\tQ7\tQ9\tQ3\tQ5\tQ7\tQ9\n'
    for option in ['', '_initEnr', '_thresUpdt', '_initEnr_thresUpdt']:
        for ratio in ['0.01', '0.1', '0.5', '0.9']:
            for enr in ['3', '5', '7', '9']:
                key = trial + '_inc_n_enr_' + enr + '_pRatio_' + ratio + option
                #if key in inc_records.keys():
                eer = inc_records[key][0]
                acc = inc_records[key][1]
                eer_string += '{:.5f}'.format(eer*100) + '\t'
                acc_string += '{:.5f}'.format(acc*100) + '\t'

        eer_string += '\n'
        acc_string += '\n'
    eer_string += '\n'
    acc_string += '\n'

with open(base_dir + 'acc_excel.txt', 'w') as f:
    f.write(acc_string)
with open(base_dir + 'eer_excel.txt', 'w') as f:
    f.write(eer_string)